# Part 1: Setup the Database

In [1]:
import sqlite3
import datetime as dt

def setup_database():
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()

    # Create tables
    mycur.execute('''
    CREATE TABLE IF NOT EXISTS bank (
        UserName TEXT PRIMARY KEY,
        Name TEXT,
        Password TEXT,
        DOB DATE,
        Address TEXT,
        Mobile_Number TEXT,
        Aadhar_no TEXT,
        Balance REAL,
        AccountType TEXT
    )
    ''')

    mycur.execute('''
    CREATE TABLE IF NOT EXISTS transactions (
        TransactionID INTEGER PRIMARY KEY AUTOINCREMENT,
        UserName TEXT,
        Date TIMESTAMP,
        Type TEXT,
        Amount REAL,
        Balance_After REAL,
        FOREIGN KEY (UserName) REFERENCES bank(UserName)
    )
    ''')

    mycon.commit()
    mycon.close()

setup_database()


In [5]:
!pip install faker

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from faker import Faker
import sqlite3
import random
import datetime as dt

fake = Faker()

def populate_bank_table():
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()
    
    account_types = ['Savings', 'Current']
    
    for _ in range(100):
        name = fake.name()
        username = fake.user_name()
        password = fake.password()
        dob = fake.date_of_birth(minimum_age=18, maximum_age=90)
        address = fake.address()
        mobile = fake.phone_number()
        aadhar = fake.ssn()
        balance = round(random.uniform(1000, 100000), 2)
        account_type = random.choice(account_types)

        mycur.execute('''
            INSERT INTO bank (UserName, Name, Password, DOB, Address, Mobile_Number, Aadhar_no, Balance, AccountType)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (username, name, password, dob, address, mobile, aadhar, balance, account_type))
    
    mycon.commit()
    mycon.close()

def populate_transactions_table():
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()

    mycur.execute("SELECT UserName FROM bank")
    users = [row[0] for row in mycur.fetchall()]

    transaction_types = ['Deposit', 'Withdrawal', 'Sent', 'Interest']

    for _ in range(100):
        username = random.choice(users)
        transaction_date = fake.date_time_between(start_date='-1y', end_date='now')
        transaction_type = random.choice(transaction_types)
        amount = round(random.uniform(100, 10000), 2)

        mycur.execute("SELECT Balance FROM bank WHERE UserName = ?", (username,))
        balance = mycur.fetchone()[0]

        if transaction_type == 'Withdrawal' and amount > balance:
            amount = balance

        new_balance = balance + amount if transaction_type in ['Deposit', 'Interest'] else balance - amount

        mycur.execute('''
            INSERT INTO transactions (UserName, Date, Type, Amount, Balance_After)
            VALUES (?, ?, ?, ?, ?)
        ''', (username, transaction_date, transaction_type, amount, new_balance))

        mycur.execute("UPDATE bank SET Balance = ? WHERE UserName = ?", (new_balance, username))

    mycon.commit()
    mycon.close()

# Populate the tables
populate_bank_table()
populate_transactions_table()


C:\Users\User\AppData\Local\Temp\ipykernel_22164\3829376103.py:25: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  mycur.execute('''
C:\Users\User\AppData\Local\Temp\ipykernel_22164\3829376103.py:56: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  mycur.execute('''


# Part 2: Create a New Account (Sign Up)

In [7]:
def create_account():
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()

    st.header("Create a New Account (Sign Up)")
    name = st.text_input("Enter Your Name")
    username = st.text_input("Enter Your UserName")
    password = st.text_input("Enter Your Password", type="password")
    today = dt.date.today()
    dob = st.date_input(
        "Enter Your Date of Birth",
        value=today - dt.timedelta(days=18*365),  # Default to 18 years ago
        min_value=dt.date(1900, 1, 1),
        max_value=today
    )
    address = st.text_area("Enter Your Address")
    mobile = st.text_input("Enter Your Mobile Number")
    aadhar = st.text_input("Enter Your Aadhar Number")
    account_type = st.selectbox("Enter Account Type", ['Savings', 'Current'])
    balance = st.number_input("Enter Initial Deposit", min_value=0.0, step=0.01)

    if st.button("Create Account"):
        if not (name and username and password and dob and address and mobile and aadhar and account_type and balance):
            st.error("Please fill out all fields.")
        else:
            dob = dob.strftime('%Y-%m-%d')
            mycur.execute('''
                INSERT INTO bank (UserName, Name, Password, DOB, Address, Mobile_Number, Aadhar_no, Balance, AccountType)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (username, name, password, dob, address, mobile, aadhar, balance, account_type))

            mycon.commit()
            mycon.close()
            st.success("Account Created Successfully!")


# Part 3: Sign In (Existing Users)

In [8]:
def sign_in():
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()

    st.header("Sign In (Existing Users)")
    username = st.text_input("Enter Your UserName")
    password = st.text_input("Enter Your Password", type="password")

    if st.button("Sign In"):
        if not (username and password):
            st.error("Please enter both username and password.")
        else:
            mycur.execute("SELECT * FROM bank WHERE UserName = ? AND Password = ?", (username, password))
            user = mycur.fetchone()

            if user:
                st.session_state['username'] = username
                st.success(f"Welcome back, {user[1]}!")
            else:
                st.error("Invalid credentials. Please try again.")

    mycon.close()


# Part 4: View Balance Status

In [9]:
def view_balance(username):
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()

    mycur.execute("SELECT Balance FROM bank WHERE UserName = ?", (username,))
    balance = mycur.fetchone()[0]

    print(f"Your current balance is: {balance} ₹")

    mycon.close()


# Part 5: View Account Details

In [10]:
def view_account_details(username):
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()

    mycur.execute("SELECT * FROM bank WHERE UserName = ?", (username,))
    user = mycur.fetchone()

    if user:
        print("\n--- Account Details ---")
        print(f"Name: {user[1]}")
        print(f"Date of Birth: {user[3]}")
        print(f"Address: {user[4]}")
        print(f"Mobile Number: {user[5]}")
        print(f"Aadhar Number: {user[6]}")
        print(f"Balance: {user[7]} ₹")
        print(f"Account Type: {user[8]}")
    else:
        print("User not found.")

    mycon.close()


# Part 6: Update Account Details

In [11]:
def update_account_details(username):
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()

    print("\n--- Update Account Details ---")
    new_address = input("Enter New Address: ")
    new_mobile = input("Enter New Mobile Number: ")

    mycur.execute('''
        UPDATE bank 
        SET Address = ?, Mobile_Number = ? 
        WHERE UserName = ?
    ''', (new_address, new_mobile, username))

    mycon.commit()
    mycon.close()

    print("Account details updated successfully!")


# Part 7: Withdraw Money

In [12]:
def withdraw_money(username):
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()

    st.header("Withdraw Money")
    amount = st.number_input("Enter the amount to withdraw", min_value=0.0, step=0.01)

    if st.button("Withdraw"):
        mycur.execute("SELECT Balance FROM bank WHERE UserName = ?", (username,))
        balance = mycur.fetchone()[0]

        if amount > balance:
            st.error("Insufficient balance.")
        else:
            new_balance = balance - amount
            mycur.execute("UPDATE bank SET Balance = ? WHERE UserName = ?", (new_balance, username))
            mycur.execute("INSERT INTO transactions (UserName, Date, Type, Amount, Balance_After) VALUES (?, ?, ?, ?, ?)", 
                          (username, dt.datetime.now(), 'Withdrawal', amount, new_balance))
            mycon.commit()
            st.success(f"Withdrawal successful. New balance is: {new_balance} ₹")

    mycon.close()


# Part 8: Deposit Money

In [13]:
def deposit_money(username):
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()

    st.header("Deposit Money")
    amount = st.number_input("Enter the amount to deposit", min_value=0.0, step=0.01)

    if st.button("Deposit"):
        mycur.execute("SELECT Balance FROM bank WHERE UserName = ?", (username,))
        balance = mycur.fetchone()[0]
        
        new_balance = balance + amount
        mycur.execute("UPDATE bank SET Balance = ? WHERE UserName = ?", (new_balance, username))
        mycur.execute("INSERT INTO transactions (UserName, Date, Type, Amount, Balance_After) VALUES (?, ?, ?, ?, ?)", 
                      (username, dt.datetime.now(), 'Deposit', amount, new_balance))
        mycon.commit()
        st.success(f"Deposit successful. New balance is: {new_balance} ₹")

    mycon.close()


# Part 9: Calculate Interest

In [14]:
def calculate_interest(username):
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()

    mycur.execute("SELECT Balance, AccountType FROM bank WHERE UserName = ?", (username,))
    balance, account_type = mycur.fetchone()

    if account_type == 'Savings':
        interest_rate = 0.04
    else:
        interest_rate = 0.02

    interest = balance * interest_rate / 12
    new_balance = balance + interest
    mycur.execute("UPDATE bank SET Balance = ? WHERE UserName = ?", (new_balance, username))
    mycur.execute("INSERT INTO transactions (UserName, Date, Type, Amount, Balance_After) VALUES (?, ?, ?, ?, ?)", 
                  (username, dt.datetime.now(), 'Interest', interest, new_balance))
    mycon.commit()
    st.success(f"Interest added. New balance is: {new_balance} ₹")

    mycon.close()


# Part 10: Transaction Report

In [15]:
def transaction_report(username):
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()

    st.header("Transaction History")
    mycur.execute("SELECT Date, Type, Amount, Balance_After FROM transactions WHERE UserName = ? ORDER BY Date DESC", (username,))
    transactions = mycur.fetchall()

    if transactions:
        st.write("<h2 style='text-align: center; color: #C05621;'>--- Transaction History ---</h2>", unsafe_allow_html=True)
        for transaction in transactions:
            st.write(f"<h3 style='text-align: center;'>Date: {transaction[0]}, Type: {transaction[1]}, Amount: {transaction[2]} ₹, Balance After: {transaction[3]} ₹</h3>", unsafe_allow_html=True)
    else:
        st.write("<h3 style='text-align: center;'>No transactions found.</h3>", unsafe_allow_html=True)

    mycon.close()

# Part 11: Send Money

In [17]:
def send_money(username):
    mycon = sqlite3.connect('banking_system.db')
    mycur = mycon.cursor()

    st.header("Send Money")
    recipient_username = st.text_input("Enter the recipient's username")
    amount = st.number_input("Enter the amount to send", min_value=0.0, step=0.01)

    if st.button("Send Money"):
        mycur.execute("SELECT UserName FROM bank WHERE UserName = ?", (recipient_username,))
        recipient = mycur.fetchone()

        if not recipient:
            st.error("Recipient not found.")
        else:
            mycur.execute("SELECT Balance FROM bank WHERE UserName = ?", (username,))
            sender_balance = mycur.fetchone()[0]

            if amount > sender_balance:
                st.error("Insufficient balance.")
            else:
                new_sender_balance = sender_balance - amount
                mycur.execute("UPDATE bank SET Balance = ? WHERE UserName = ?", (new_sender_balance, username))
                mycur.execute("INSERT INTO transactions (UserName, Date, Type, Amount, Balance_After) VALUES (?, ?, ?, ?, ?)", 
                              (username, dt.datetime.now(), 'Transfer Out', amount, new_sender_balance))
                
                mycur.execute("SELECT Balance FROM bank WHERE UserName = ?", (recipient_username,))
                recipient_balance = mycur.fetchone()[0]
                new_recipient_balance = recipient_balance + amount
                mycur.execute("UPDATE bank SET Balance = ? WHERE UserName = ?", (new_recipient_balance, recipient_username))
                mycur.execute("INSERT INTO transactions (UserName, Date, Type, Amount, Balance_After) VALUES (?, ?, ?, ?, ?)", 
                              (recipient_username, dt.datetime.now(), 'Transfer In', amount, new_recipient_balance))

                mycon.commit()
                st.success(f"Money sent successfully! Your new balance is: {new_sender_balance} ₹")

    mycon.close()


# Part 12: Menu Function

# RICH BANK OF INDIA (RBI) - Banking System

Welcome to the **RICH BANK OF INDIA** (RBI) banking system project. This project is a simple banking application built using Streamlit, SQLite, and Python. It allows users to create accounts, sign in, and perform various banking operations through a web interface.

## Features

- **Sign Up / Sign In**: Create a new account or sign in to an existing account.
- **View Balance**: Check the current balance of your account.
- **View Account Details**: View detailed information about your account.
- **Update Account Details**: Update your address and mobile number.
- **Withdraw Money**: Withdraw funds from your account.
- **Deposit Money**: Deposit funds into your account.
- **Calculate Interest**: Calculate and add interest to your savings account.
- **Transaction Report**: Generate a report of transactions within a specified date range.
- **Send Money**: Transfer funds to another user's account.

## Installation

To run this project, you need Python and Streamlit installed. Follow the steps below to set up the project:

1. **Clone the repository:**
   ```bash
   git clone https://github.com/Nimish-Sikka/rich-bank-of-india.git
   cd rich-bank-of-india


In [19]:
def menu():
    while True:
        print("\n--- Banking System Menu ---")
        print("1. Create a New Account (Sign Up)")
        print("2. Sign In (Existing Users)")
        print("3. View Balance Status")
        print("4. View Account Details")
        print("5. Update Account Details")
        print("6. Withdraw Money")
        print("7. Deposit Money")
        print("8. Calculate Interest")
        print("9. Transaction Report")
        print("10. Send Money")
        print("0. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            create_account()
        elif choice == '2':
            username = sign_in()
        elif choice == '3':
            if username:
                view_balance(username)
            else:
                print("Please sign in first.")
        elif choice == '4':
            if username:
                view_account_details(username)
            else:
                print("Please sign in first.")
        elif choice == '5':
            if username:
                update_account_details(username)
            else:
                print("Please sign in first.")
        elif choice == '6':
            if username:
                withdraw_money(username)
            else:
                print("Please sign in first.")
        elif choice == '7':
            if username:
                deposit_money(username)
            else:
                print("Please sign in first.")
        elif choice == '8':
            if username:
                calculate_interest(username)
            else:
                print("Please sign in first.")
        elif choice == '9':
            if username:
                transaction_report(username)
            else:
                print("Please sign in first.")
        elif choice == '10':
            if username:
                send_money(username)
            else:
                print("Please sign in first.")
        elif choice == '0':
            print("Thank you for using the banking system. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

# Run the menu
menu()



--- Banking System Menu ---
1. Create a New Account (Sign Up)
2. Sign In (Existing Users)
3. View Balance Status
4. View Account Details
5. Update Account Details
6. Withdraw Money
7. Deposit Money
8. Calculate Interest
9. Transaction Report
10. Send Money
0. Exit
Welcome back, Nimish Sikka!

--- Banking System Menu ---
1. Create a New Account (Sign Up)
2. Sign In (Existing Users)
3. View Balance Status
4. View Account Details
5. Update Account Details
6. Withdraw Money
7. Deposit Money
8. Calculate Interest
9. Transaction Report
10. Send Money
0. Exit
Your current balance is: 201334.44444444444 ₹

--- Banking System Menu ---
1. Create a New Account (Sign Up)
2. Sign In (Existing Users)
3. View Balance Status
4. View Account Details
5. Update Account Details
6. Withdraw Money
7. Deposit Money
8. Calculate Interest
9. Transaction Report
10. Send Money
0. Exit
Invalid choice. Please try again.

--- Banking System Menu ---
1. Create a New Account (Sign Up)
2. Sign In (Existing Users)
3. 